### **Extracción Data - Autos**

In [150]:
## Librerias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from tqdm import tqdm  # pip install tqdm
import datetime as dt
import re, time


In [83]:
# enlace = "https://neoauto.com/venta-de-autos-seminuevos"
enlace = "https://neoauto.com/venta-de-autos-nuevos?page=3"
# enlace = "https://neoauto.com/venta-de-autos-nuevos?page=1"

In [3]:
# Generate a synthetic dataset for R practice (>=200 rows) and save as CSV

import numpy as np
import pandas as pd

rng = np.random.default_rng(42)
n = 300  # number of records

# Company IDs
empresa_id = np.arange(1, n + 1)

# Sector: 1=servicios, 2=industria, 3=comercio (weighted to be varied)
sector = rng.choice([1, 2, 3], size=n, p=[0.45, 0.30, 0.25])

# Construct ventas with a 3-component mixture so you have values in buckets:
# A: 0-999.99, B: 1000-1999.99, C: 2000+
mix = rng.choice(["A", "B", "C"], size=n, p=[0.4, 0.3, 0.3])

ventas = np.empty(n, dtype=float)
mask_a = mix == "A"
mask_b = mix == "B"
mask_c = mix == "C"

ventas[mask_a] = rng.uniform(200, 980, size=mask_a.sum())
ventas[mask_b] = rng.uniform(1000, 1990, size=mask_b.sum())
ventas[mask_c] = rng.uniform(2000, 8000, size=mask_c.sum())

# Round to 2 decimals
ventas = np.round(ventas, 2)

# Empleados ~ depends weakly on ventas
empleados = np.maximum(1, rng.poisson(lam=np.clip(ventas/500, 2, 50))).astype(int)

# Tamaño según empleados (simple rule)
def clasifica_tamano(e):
    if e < 10:
        return "micro"
    elif e < 50:
        return "pequeña"
    elif e < 250:
        return "mediana"
    else:
        return "grande"

tamano = pd.Series(empleados).apply(clasifica_tamano)

# Margen (%) y utilidad = ventas * margen
margen_pct = np.round(rng.normal(loc=0.18, scale=0.07, size=n), 3)  # allow some negatives
utilidad = np.round(ventas * margen_pct, 2)

# Antigüedad en años
antiguedad = rng.integers(0, 60, size=n)

# Provincia (solo para colorcito)
provincias = ["Lima", "Arequipa", "Cusco", "La Libertad", "Piura", "Junín", "Lambayeque"]
provincia = rng.choice(provincias, size=n, replace=True)

# Build DataFrame
df = pd.DataFrame({
    "ID_EMPRESA": empresa_id,
    "SECTOR": sector,               # numérica para recode en R
    "VENTAS": ventas,               # cuantitativa para cortes A/B/C
    "EMPLEADOS": empleados,
    "TAMANO": tamano,               # categórica para tablas cruzadas
    "MARGEN_PCT": margen_pct,
    "UTILIDAD": utilidad,
    "ANTIGUEDAD": antiguedad,
    "PROVINCIA": provincia,
})

# Save to CSV
# path = "/mnt/data/empresas.csv"
df.to_excel("empresas.xlsx", index=False)



## Driver Setup

In [84]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")   # modo oculto
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
# options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options = options)
# driver.minimize_window()
driver.get(enlace)

In [85]:
resultados = driver.find_element(By.CLASS_NAME, "s-results__count")
print(resultados.text)

118 Resultados


extraccion de datos

In [86]:
cards = 'Hyundai Accent 2024\nAUTO NUEVO'
        #  'Hyundai Creta Grand 2025\nAUTO NUEVO',
        #  'Hyundai Tucson 2024\nAUTO NUEVO',
        #  'Hyundai Tucson 2024\nAUTO NUEVO',
        #  'Hyundai Creta Grand 2025\nAUTO NUEVO',
        #  'Ford F-150 2024\nAUTO NUEVO',
        #  'Hyundai Grand I10 2024\nAUTO NUEVO',
        #  'Hyundai Tucson 2025\nAUTO NUEVO'
         

tarejtas = cards.split("\n", 1)[0]
tarejtas

'Hyundai Accent 2024'

In [87]:
# BRANDS = [
#     "Alfa Romeo","Aston Martin","Land Rover","Mercedes Benz","Great Wall",
#     "Volkswagen","Chevrolet","Toyota","Hyundai","Kia","Nissan","BMW","Audi",
#     "Peugeot","Renault","Mazda","Subaru","Mitsubishi","Suzuki","Haval","JAC",
#     "Changan","Geely","DFSK","Chery","Volvo","Porsche","Jaguar","Jeep","Ram",
#     "Dodge","Ford","Seat","Skoda","Mini","BYD","MG","GMC","Lexus","Citroën","Citroen"
# ]

In [90]:
encabezado = driver.find_elements(By.CLASS_NAME, "c-results__header")

titulo_auto = []

for enc in encabezado:
    titulo = enc.text
    titulo_ = titulo.split("\n", 1)[0].strip()
    titulo_auto.append(titulo_)
    
    
    
titulo_auto

['Mazda 2 2024',
 'Renault Stepway 2025',
 'Dfsk Glory 560 2026',
 'Honda Pilot 2025',
 'Mazda Bt-50 2026',
 'Ford Expedition 2025',
 'Honda Hr-V 2025',
 'Honda City 2024',
 'Kia Sportage 2025',
 'Mazda 3 Sport 2024',
 'Dfsk D1 2024',
 'Kia Sportage 2025',
 'Nissan Sentra 2024',
 'Kia Sportage 2024',
 'Jmc Grand Avenue Luxury 2026',
 'Renault Duster 2025',
 'Dfsk C37 Minibus 2025',
 'Hyundai Grand I10 2026',
 'Kia Sportage 2024',
 'Ford Bronco Sport 2025']

In [94]:
BRANDS = [
    "Alfa Romeo","Aston Martin","Land Rover","Mercedes Benz","Great Wall",
    "Volkswagen","Chevrolet","Toyota","Hyundai","Kia","Nissan","BMW","Audi",
    "Peugeot","Renault","Mazda","Subaru","Mitsubishi","Suzuki","Haval","JAC",
    "Changan","Geely","DFSK","Chery","Volvo","Porsche","Jaguar","Jeep","Ram",
    "Dodge","Ford","Seat","Skoda","Mini","BYD","MG","GMC","Lexus","Citroën","Citroen",
    "Honda","JMC"
]

titulos = [
 'Mazda 2 2024',
 'Renault Stepway 2025',
 'Dfsk Glory 560 2026',
 'Honda Pilot 2025',
 'Mazda Bt-50 2026',
 'Ford Expedition 2025',
 'Honda Hr-V 2025',
 'Honda City 2024',
 'Kia Sportage 2025',
 'Mazda 3 Sport 2024',
 'Dfsk D1 2024',
 'Kia Sportage 2025',
 'Nissan Sentra 2024',
 'Kia Sportage 2024',
 'Jmc Grand Avenue Luxury 2026',
 'Renault Duster 2025',
 'Dfsk C37 Minibus 2025',
 'Hyundai Grand I10 2026',
 'Kia Sportage 2024',
 'Ford Bronco Sport 2025'
]

titulo_auto, marca_auto, modelo_auto, año_auto = [], [], [], []

for t in titulos:
    # 1. Buscar año con regex (1980–2035)
    m = re.search(r'\b(19[8-9]\d|20[0-3]\d)\b', t)
    anio = m.group(0) if m else None
    
    # 2. Marcas: buscar la que aparezca en el título (case-insensitive)
    marca = next((b for b in BRANDS if b.lower() in t.lower()), "Otra Marca")
    
    # 3. Modelo = título quitando marca y año
    modelo = t
    if marca != "Otra Marca":
        modelo = re.sub(marca, "", modelo, flags=re.IGNORECASE).strip()
    if anio:
        modelo = modelo.replace(anio, "").strip()
    
    # Guardamos
    titulo_auto.append(t)
    marca_auto.append(marca)
    modelo_auto.append(modelo)
    año_auto.append(anio)

for i in range(len(titulos)):
    print(f"{titulo_auto[i]} -> Marca: {marca_auto[i]}, Modelo: {modelo_auto[i]}, Año: {año_auto[i]}")

Mazda 2 2024 -> Marca: Mazda, Modelo: 2, Año: 2024
Renault Stepway 2025 -> Marca: Renault, Modelo: Stepway, Año: 2025
Dfsk Glory 560 2026 -> Marca: DFSK, Modelo: Glory 560, Año: 2026
Honda Pilot 2025 -> Marca: Honda, Modelo: Pilot, Año: 2025
Mazda Bt-50 2026 -> Marca: Mazda, Modelo: Bt-50, Año: 2026
Ford Expedition 2025 -> Marca: Ford, Modelo: Expedition, Año: 2025
Honda Hr-V 2025 -> Marca: Honda, Modelo: Hr-V, Año: 2025
Honda City 2024 -> Marca: Honda, Modelo: City, Año: 2024
Kia Sportage 2025 -> Marca: Kia, Modelo: Sportage, Año: 2025
Mazda 3 Sport 2024 -> Marca: Mazda, Modelo: 3 Sport, Año: 2024
Dfsk D1 2024 -> Marca: DFSK, Modelo: D1, Año: 2024
Kia Sportage 2025 -> Marca: Kia, Modelo: Sportage, Año: 2025
Nissan Sentra 2024 -> Marca: Nissan, Modelo: Sentra, Año: 2024
Kia Sportage 2024 -> Marca: Kia, Modelo: Sportage, Año: 2024
Jmc Grand Avenue Luxury 2026 -> Marca: JMC, Modelo: Grand Avenue Luxury, Año: 2026
Renault Duster 2025 -> Marca: Renault, Modelo: Duster, Año: 2025
Dfsk C37 M

In [71]:

BRANDS = [
    "Alfa Romeo","Aston Martin","Land Rover","Mercedes Benz","Great Wall",
    "Volkswagen","Chevrolet","Toyota","Hyundai","Kia","Nissan","BMW","Audi",
    "Peugeot","Renault","Mazda","Subaru","Mitsubishi","Suzuki","Haval","JAC",
    "Changan","Geely","DFSK","Chery","Volvo","Porsche","Jaguar","Jeep","Ram",
    "Dodge","Ford","Seat","Skoda","Mini","BYD","MG","GMC","Lexus","Citroën","Citroen"
]

encabezado = driver.find_elements(By.CLASS_NAME, "c-results__header")

titulo_auto = []
marca_auto = []
modelo_auto = []
año_auto = []

for enc in encabezado:
    titulo = enc.text
    titulo_ = titulo.split("\n", 1)[0].strip()
    for brand in BRANDS:
        if brand in titulo_:
            
            año = titulo_.split()[-1]
            modelo = titulo_.replace(brand, "").replace(año, "").strip()

            titulo_auto.append(titulo_)
            marca_auto.append(brand)    
            modelo_auto.append(modelo)
            año_auto.append(año)
        else:
            brand = "Otra Marca"
titulo_auto         

['Mazda 2 2024',
 'Renault Stepway 2025',
 'Mazda Bt-50 2026',
 'Ford Expedition 2025',
 'Kia Sportage 2025',
 'Mazda 3 Sport 2024',
 'Kia Sportage 2025',
 'Nissan Sentra 2024',
 'Kia Sportage 2024',
 'Renault Duster 2025',
 'Dfsk C37 Minibus 2025',
 'Hyundai Grand I10 2026',
 'Kia Sportage 2024',
 'Ford Bronco Sport 2025']

In [49]:
import re

BRANDS = [
    "Alfa Romeo","Aston Martin","Land Rover","Mercedes Benz","Great Wall",
    "Volkswagen","Chevrolet","Toyota","Hyundai","Kia","Nissan","BMW","Audi",
    "Peugeot","Renault","Mazda","Subaru","Mitsubishi","Suzuki","Haval","JAC",
    "Changan","Geely","DFSK","Chery","Volvo","Porsche","Jaguar","Jeep","Ram",
    "Dodge","Ford","Seat","Skoda","Mini","BYD","MG","GMC","Lexus","Citroën","Citroen"
]
brand_re = re.compile(r'^(?P<brand>' + '|'.join(map(re.escape, sorted(BRANDS, key=len, reverse=True))) + r')\b', re.I)
year_re  = re.compile(r'((?:19|20)\d{2})\s*$')
KEEP_UPPER = {"BMW","BYD","GMC","MG","RAM","DFSK"}

def parse_card_title(raw: str):
    # 1) Quédate con la primera línea (títulos vienen como "… 2024\nAUTO NUEVO")
    title = raw.split("\n", 1)[0].strip()

    # 2) Año al final
    m_year = year_re.search(title)
    anio = int(m_year.group(1)) if m_year else None
    name = year_re.sub("", title).strip()

    # 3) Marca al inicio (incluye marcas de 2 palabras)
    m_brand = brand_re.match(name)
    if m_brand:
        brand_raw = m_brand.group("brand")
        marca = brand_raw if brand_raw.upper() in KEEP_UPPER else brand_raw.title()
        modelo = name[m_brand.end():].strip()
    else:
        parts = name.split(maxsplit=1)
        marca = (parts[0].upper() if parts and parts[0].upper() in KEEP_UPPER
                 else (parts[0].title() if parts else ""))
        modelo = parts[1] if len(parts) > 1 else ""

    return {"titulo": title, "marca": marca, "modelo": modelo, "anio": anio}

cards = ['Hyundai Accent 2024\nAUTO NUEVO',
         'Hyundai Creta Grand 2025\nAUTO NUEVO',
         'Hyundai Tucson 2024\nAUTO NUEVO',
         'Hyundai Tucson 2024\nAUTO NUEVO',
         'Hyundai Creta Grand 2025\nAUTO NUEVO',
         'Ford F-150 2024\nAUTO NUEVO',
         'Hyundai Grand I10 2024\nAUTO NUEVO',
         'Hyundai Tucson 2025\nAUTO NUEVO']

parsed = [parse_card_title(s) for s in cards]
for r in parsed:
    print(r)

{'titulo': 'Hyundai Accent 2024', 'marca': 'Hyundai', 'modelo': 'Accent', 'anio': 2024}
{'titulo': 'Hyundai Creta Grand 2025', 'marca': 'Hyundai', 'modelo': 'Creta Grand', 'anio': 2025}
{'titulo': 'Hyundai Tucson 2024', 'marca': 'Hyundai', 'modelo': 'Tucson', 'anio': 2024}
{'titulo': 'Hyundai Tucson 2024', 'marca': 'Hyundai', 'modelo': 'Tucson', 'anio': 2024}
{'titulo': 'Hyundai Creta Grand 2025', 'marca': 'Hyundai', 'modelo': 'Creta Grand', 'anio': 2025}
{'titulo': 'Ford F-150 2024', 'marca': 'Ford', 'modelo': 'F-150', 'anio': 2024}
{'titulo': 'Hyundai Grand I10 2024', 'marca': 'Hyundai', 'modelo': 'Grand I10', 'anio': 2024}
{'titulo': 'Hyundai Tucson 2025', 'marca': 'Hyundai', 'modelo': 'Tucson', 'anio': 2025}


In [50]:
precio = driver.find_elements(By.CLASS_NAME, "c-results-mount__price")
precio_auto = []
for prec in precio:
    costo = prec.text
    precio_auto.append(costo)
    
precio_auto[0]

'US$ 20,990'

In [51]:
tag =  driver.find_elements(By.CLASS_NAME, "c-results-tag__stick")
tag_ = []
for det in tag:
    descripcion = det.text
    tag_.append(descripcion)

tag_[0]

'PREMIUM'

In [52]:

detalles = driver.find_elements(By.CLASS_NAME, "c-results-details__description")

detalle_auto = []
combustible_auto = []
transmision_auto = []
caja_auto = []
kilometraje_auto = []
ubicacion_auto = []
descripcion_short_auto = []

detalles_autos = []

for detalle in detalles:
    
    detalle_text = detalle.text
    # print(info)
    
    # normaliza y separa líneas no vacías
    t = re.sub(r'[\r\xa0]', ' ', detalle_text).strip()
    lines = [l.strip() for l in t.splitlines() if l.strip()]

    # 1) combustible y transmisión (vienen en la 1ra línea separados por "|")
    combustible = transmision_raw = None
    if lines:
        p0 = re.split(r'\s*\|\s*', lines[0])  # acepta con o sin espacios: "Gasolina|Automática - Secuencial"
        combustible = p0[0].title() if p0 else None
        transmision_raw = p0[1] if len(p0) > 1 else None

    # 1.1) separa transmisión en tipo y caja si viene "Automática - Secuencial"
    tipo_transmision = caja = None
    if transmision_raw:
        parts = [p.strip() for p in transmision_raw.split("-")]
        tipo_transmision = parts[0]
        caja = parts[1] if len(parts) > 1 else None

    # 2) kilometraje (busca en todo el texto)
    km_m = re.search(r'(\d[\d.,]*)\s*kms?', t, flags=re.I)
    kilometraje_km = int(re.sub(r'[^\d]', '', km_m.group(1))) if km_m else None

    # 3) ubicación (primera línea con coma que no sea "Kms")
    ubicacion = next((l for l in lines if ',' in l and 'km' not in l.lower()), None)

    # 4) “modelo/etiqueta” (última línea)
    modelo_line = lines[-1] if lines else None
    
    # salidas
    detalle_auto.append(detalle_text)
    combustible_auto.append(combustible)
    transmision_auto.append(tipo_transmision)
    caja_auto.append(caja)
    kilometraje_auto.append(kilometraje_km)
    ubicacion_auto.append(ubicacion)
    
    detalles_autos.append({
        "detalle": detalle_text,
        "combustible": combustible,
        "transmision": tipo_transmision,
        "caja": caja,
        "kilometraje_km": kilometraje_km,
        "ubicacion": ubicacion,
        "descripcion_short": modelo_line
    })
#     descripcion_short_auto.append(modelo_line)

detalles_autos

[{'detalle': 'Gasolina|Mecánica\nLima, Lima\nCompra Tu Próximo 0KM En Maquinarias Mazda',
  'combustible': 'Gasolina',
  'transmision': 'Mecánica',
  'caja': None,
  'kilometraje_km': 0,
  'ubicacion': 'Lima, Lima',
  'descripcion_short': 'Compra Tu Próximo 0KM En Maquinarias Mazda'},
 {'detalle': 'Gasolina|Mecánica\nLima, Lima\nCompra Tu Próximo 0KM En Maquinarias Renault',
  'combustible': 'Gasolina',
  'transmision': 'Mecánica',
  'caja': None,
  'kilometraje_km': 0,
  'ubicacion': 'Lima, Lima',
  'descripcion_short': 'Compra Tu Próximo 0KM En Maquinarias Renault'},
 {'detalle': 'Gasolina|Automática\nLima, Lima\nCompra Tu Próximo 0KM En Maquinarias DFSK',
  'combustible': 'Gasolina',
  'transmision': 'Automática',
  'caja': None,
  'kilometraje_km': 0,
  'ubicacion': 'Lima, Lima',
  'descripcion_short': 'Compra Tu Próximo 0KM En Maquinarias DFSK'},
 {'detalle': 'Gasolina|Automática\nLima, Lima\nCompra Tu Próximo 0KM En Maquinarias Honda',
  'combustible': 'Gasolina',
  'transmision'

In [53]:
enlaces = []
enlace = driver.find_elements(By.CLASS_NAME, "c-results__link")   
for link in enlace:
    url = link.get_attribute("href")    
    enlaces.append(url)

enlaces

['https://neoauto.com/auto/nuevo/mazda-2-2024-1827037',
 'https://neoauto.com/auto/nuevo/renault-stepway-2025-1705791',
 'https://neoauto.com/auto/nuevo/dfsk-glory-560-2026-1827069',
 'https://neoauto.com/auto/nuevo/honda-pilot-2025-1718653',
 'https://neoauto.com/auto/nuevo/mazda-bt-50-2026-1705782',
 'https://neoauto.com/auto/nuevo/ford-expedition-2025-1743700',
 'https://neoauto.com/auto/nuevo/honda-hr-v-2025-1769746',
 'https://neoauto.com/auto/nuevo/honda-city-2024-1843626',
 'https://neoauto.com/auto/nuevo/kia-sportage-2025-1846407',
 'https://neoauto.com/auto/nuevo/mazda-3-sport-2024-1705781',
 'https://neoauto.com/auto/nuevo/dfsk-d1-2024-1827064',
 'https://neoauto.com/auto/nuevo/kia-sportage-2025-1846406',
 'https://neoauto.com/auto/nuevo/nissan-sentra-2024-1709645',
 'https://neoauto.com/auto/nuevo/kia-sportage-2024-1846340',
 'https://neoauto.com/auto/nuevo/jmc-grand-avenue-luxury-2026-1842721',
 'https://neoauto.com/auto/nuevo/renault-duster-2025-1705784',
 'https://neoauto

En resumen se tiene

In [41]:
titulo_auto
# marca_auto
# modelo_auto
# año_auto

['Mazda 2 2024',
 'Renault Stepway 2025',
 'Mazda Bt-50 2026',
 'Ford Expedition 2025',
 'Kia Sportage 2025',
 'Mazda 3 Sport 2024',
 'Kia Sportage 2025',
 'Nissan Sentra 2024',
 'Kia Sportage 2024',
 'Renault Duster 2025',
 'Dfsk C37 Minibus 2025',
 'Hyundai Grand I10 2026',
 'Kia Sportage 2024',
 'Ford Bronco Sport 2025']

In [54]:
precio_auto

['US$ 20,990',
 'US$ 16,490',
 'US$ 16,190',
 'US$ 57,990',
 'US$ 30,590',
 'US$ 83,990',
 'US$ 25,990',
 'US$ 32,990',
 'US$ 30,990',
 'US$ 26,990',
 'US$ 21,490',
 'US$ 30,990',
 'US$ 18,190',
 'US$ 29,990',
 'US$ 23,990',
 'US$ 19,490',
 'US$ 12,790',
 'US$ 12,490',
 'US$ 29,990',
 'US$ 49,490']

In [96]:
data = pd.DataFrame({
    "titulo": titulo_auto, "modelo": modelo_auto, "año": año_auto, "precio": precio_auto, "detalle": detalle_auto, "enlace": enlaces,})

data

,titulo,modelo,año,precio,detalle,enlace
0,Mazda 2 2024,2,2024,"US$ 20,990","Gasolina|Mecánica\nLima, Lima\nCompra Tu Próxi...",https://neoauto.com/auto/nuevo/mazda-2-2024-18...
1,Renault Stepway 2025,Stepway,2025,"US$ 16,490","Gasolina|Mecánica\nLima, Lima\nCompra Tu Próxi...",https://neoauto.com/auto/nuevo/renault-stepway...
2,Dfsk Glory 560 2026,Glory 560,2026,"US$ 16,190","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",https://neoauto.com/auto/nuevo/dfsk-glory-560-...
3,Honda Pilot 2025,Pilot,2025,"US$ 57,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",https://neoauto.com/auto/nuevo/honda-pilot-202...
4,Mazda Bt-50 2026,Bt-50,2026,"US$ 30,590","Gasolina|Mecánica\nLima, Lima\nCompra Tu Próxi...",https://neoauto.com/auto/nuevo/mazda-bt-50-202...
5,Ford Expedition 2025,Expedition,2025,"US$ 83,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",https://neoauto.com/auto/nuevo/ford-expedition...
6,Honda Hr-V 2025,Hr-V,2025,"US$ 25,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",https://neoauto.com/auto/nuevo/honda-hr-v-2025...
7,Honda City 2024,City,2024,"US$ 32,990","Gasolina|Automática\nLima, Lima\nVehículos 0KM...",https://neoauto.com/auto/nuevo/honda-city-2024...
8,Kia Sportage 2025,Sportage,2025,"US$ 30,990","Gasolina|Automática - Secuencial\nLima, Lima\n...",https://neoauto.com/auto/nuevo/kia-sportage-20...
9,Mazda 3 Sport 2024,3 Sport,2024,"US$ 26,990","Gasolina|Mecánica\nLima, Lima\nCompra Tu Próxi...",https://neoauto.com/auto/nuevo/mazda-3-sport-2...


In [58]:
detalle_auto
combustible_auto
transmision_auto
caja_auto
kilometraje_auto
ubicacion_auto


['Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima',
 'Lima, Lima']

In [55]:
enlaces


['https://neoauto.com/auto/nuevo/mazda-2-2024-1827037',
 'https://neoauto.com/auto/nuevo/renault-stepway-2025-1705791',
 'https://neoauto.com/auto/nuevo/dfsk-glory-560-2026-1827069',
 'https://neoauto.com/auto/nuevo/honda-pilot-2025-1718653',
 'https://neoauto.com/auto/nuevo/mazda-bt-50-2026-1705782',
 'https://neoauto.com/auto/nuevo/ford-expedition-2025-1743700',
 'https://neoauto.com/auto/nuevo/honda-hr-v-2025-1769746',
 'https://neoauto.com/auto/nuevo/honda-city-2024-1843626',
 'https://neoauto.com/auto/nuevo/kia-sportage-2025-1846407',
 'https://neoauto.com/auto/nuevo/mazda-3-sport-2024-1705781',
 'https://neoauto.com/auto/nuevo/dfsk-d1-2024-1827064',
 'https://neoauto.com/auto/nuevo/kia-sportage-2025-1846406',
 'https://neoauto.com/auto/nuevo/nissan-sentra-2024-1709645',
 'https://neoauto.com/auto/nuevo/kia-sportage-2024-1846340',
 'https://neoauto.com/auto/nuevo/jmc-grand-avenue-luxury-2026-1842721',
 'https://neoauto.com/auto/nuevo/renault-duster-2025-1705784',
 'https://neoauto

In [97]:
len(titulo_auto)

20

In [61]:
len(precio_auto)

20

In [60]:
len(detalle_auto)

20

In [59]:
len(enlaces)

20

## Extraccion de Informacion de segun el tipo de Vehicuklo y Condicion

Es necesario preguntar si quiere un auto, nuevo, usado o semenuevo.

In [ ]:
while True:
    tipo = input("Es necesario preguntar si quiere un auto, autos (1), motos (2) o camiones (3)")

    if tipo == "1":
        tipo_vehiculo = "autos"
        break
    elif tipo == "2":
        tipo_vehiculo = "motos"
        break
    elif tipo == "3":
        tipo_vehiculo = "camiones"  
        break  
    
while True:
    tipo = input("Es necesario preguntar si quiere un auto que tipo de auto: camioneta (1), sedan (2), hatchback (3), pick-up (4), vans (5)")

    if tipo == "1":
        categoria = "camionetas-suv"
        break
    elif tipo == "2":
        categoria = "sedan"
        break
    elif tipo == "3":
        categoria = "hatchback" 
        break  
    elif tipo == "4":
        categoria = "pick-up" 
        break  
    elif tipo == "5":
        categoria = "vans" 
        break  
    elif tipo == "6":
        categoria = "deportivo" 
        break  


while True:
    tipo = input("Es necesario preguntar si quiere un auto, nuevo (1), usados (2) o seminuevos (3)")

    unidad = ""
    if tipo == "1":
        unidad = "nuevos"
        break
    elif tipo == "2":
        unidad = "usados"
        break
    elif tipo == "3":
        unidad = "seminuevos"
        break  

print(f"Se extraerán datos de: {tipo_vehiculo} - {unidad}")

Se extraerán datos de: autos - nuevos


Construccion de enlace

## Extraccion de Datos

In [99]:
# usados = range(1, 182)
# nuevo = range(1, 6)
# seminuevos = range(1, 44)

paginas = []

if unidad == "usados":
    for i in  range(1, 182):
        url = f"https://neoauto.com/venta-de-{tipo_vehiculo}-{unidad}?page={i}"
        paginas.append({
            "url": url,
            "tipo": unidad,
            "tipo_vehiculo": tipo_vehiculo
        })

if unidad == "nuevos":
    for i in  range(1, 7):
        url = f"https://neoauto.com/venta-de-{tipo_vehiculo}-{unidad}?page={i}"
        paginas.append({
            "url": url,
            "tipo": unidad,
            "tipo_vehiculo": tipo_vehiculo
        })

if unidad == "seminuevos":
    for i in  range(1, 44):
        url = f"https://neoauto.com/venta-de-{tipo_vehiculo}-{unidad}?page={i}"
        paginas.append({
            "url": url,
            "tipo": unidad,
            "tipo_vehiculo": tipo_vehiculo
        })


# paginas[:10]
print(f"Se extraerán datos de {len(paginas)} páginas")

Se extraerán datos de 6 páginas


In [131]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")   # modo oculto
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
# options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options = options)
# driver.minimize_window()


neo_autos = []

for enlace in tqdm(paginas, desc="Páginas procesadas", unit="vehiculo"):
    
    tipo = enlace["tipo"]
    link_pagina = enlace["url"]
    tipo_vehiculo = enlace["tipo_vehiculo"]
    
    driver.get(link_pagina)
    
    BRANDS = sorted([
        # Europa
        "Alfa Romeo","Aston Martin","Audi","Bentley","BMW","Bugatti","Citroen","Cupra",
        "Dacia","Ferrari","Fiat","Jaguar","Lamborghini","Land Rover","Maserati","Maybach",
        "McLaren","Mercedes Benz","Mini","Opel","Peugeot","Porsche","Renault","Rolls Royce",
        "Seat","Skoda","Smart","Volkswagen","Volvo",
    
        # Japón
        "Acura","Daihatsu","Honda","Infiniti","Isuzu","Lexus","Mazda","Mitsubishi",
        "Nissan","Subaru","Suzuki","Toyota",
    
        # Corea
        "Hyundai","Genesis","Kia","SsangYong",
    
        # Estados Unidos
        "Buick","Cadillac","Chevrolet","Chrysler","Dodge","Ford","GMC","Hummer","Jeep",
        "Lincoln","Ram","Tesla",
    
        # China (muy usados en Latam)
        "BAIC","BYD","Changan","Chery","DFSK","Dongfeng","FAW","Foton","Geely","Great Wall",
        "Haval","JAC","Jetour","Maxus","MG","Omoda","Zotye", "JMC",
    
        # India
        "Mahindra","Tata",
    
        # Reino Unido
        "Lotus","Morgan","Noble","Vauxhall",
    
        # Otros/Exóticos
        "Abarth","Apollo","Arash","Arrinera","Artega","Brilliance","Caterham","DeLorean",
        "Denza","Fisker","Koenigsegg","Lucid","Pagani","Polestar","Rimac","Rover","Saab",
        "Scion","Spyker","Weismann","Zenvo"
    ], key=len, reverse=True)


    ## NOMBRE, MARCA, MODELO, AÑO
    
    encabezado = driver.find_elements(By.CLASS_NAME, "c-results__header")
    
    # Regex para ruidos (ej. AUTO NUEVO, NUEVA, etc.)
    NOISE_RX = re.compile(r'\bauto\s+nuev[oa]s?\b', re.I)

    titulo_auto, marca_auto, modelo_auto, año_auto = [], [], [], []

    for titulo in encabezado:

        # 1. Limpieza de ruido
        t = NOISE_RX.sub("", titulo.text).strip()

        # 1. Buscar año con regex (1980–2035)
        m = re.search(r'\b(19[8-9]\d|20[0-3]\d)\b', t)
        anio = m.group(0) if m else None

        # 2. Marcas: buscar la que aparezca en el título (case-insensitive)
        marca = next((b for b in BRANDS if b.lower() in t.lower()), "Otra Marca")

        # 3. Modelo = título quitando marca y año
        modelo = t
        if marca != "Otra Marca":
            modelo = re.sub(marca, "", modelo, flags=re.IGNORECASE).strip()
        if anio:
            modelo = modelo.replace(anio, "").strip()

        # Guardamos
        titulo_auto.append(t)
        marca_auto.append(marca)
        modelo_auto.append(modelo)
        año_auto.append(anio)

    
    ## PRECIO
    
    precio = driver.find_elements(By.CLASS_NAME, "c-results-mount__price")
    precio_auto = []
    for prec in precio:
        costo = prec.text
        precio_auto.append(costo)

    precio_auto[0]
    
    ## TAGS
    
    tag =  driver.find_elements(By.CLASS_NAME, "c-results-tag__stick")
    tags = []
    for det in tag:
        descripcion = det.text
        tags.append(descripcion)
    
    ## DETALLE DE AUTO
        
    detalles = driver.find_elements(By.CLASS_NAME, "c-results-details__description")

    detalle_auto = []
    combustible_auto = []
    transmision_auto = []
    caja_auto = []
    kilometraje_auto = []
    ubicacion_auto = []

    for detalle in detalles:

        detalle_text = detalle.text

        # normaliza y separa líneas no vacías
        t = re.sub(r'[\r\xa0]', ' ', detalle_text).strip()
        lines = [l.strip() for l in t.splitlines() if l.strip()]

        # 1) combustible y transmisión (vienen en la 1ra línea separados por "|")
        combustible = transmision_raw = None
        if lines:
            p0 = re.split(r'\s*\|\s*', lines[0])  # acepta con o sin espacios: "Gasolina|Automática - Secuencial"
            combustible = p0[0].title() if p0 else None
            transmision_raw = p0[1] if len(p0) > 1 else None

        # 1.1) separa transmisión en tipo y caja si viene "Automática - Secuencial"
        tipo_transmision = caja = None
        if transmision_raw:
            parts = [p.strip() for p in transmision_raw.split("-")]
            tipo_transmision = parts[0]
            caja = parts[1] if len(parts) > 1 else None

        # 2) kilometraje (busca en todo el texto)
        km_m = re.search(r'(\d[\d.,]*)\s*kms?', t, flags=re.I)
        kilometraje_km = int(re.sub(r'[^\d]', '', km_m.group(1))) if km_m else None

        # 3) ubicación (primera línea con coma que no sea "Kms")
        ubicacion = next((l for l in lines if ',' in l and 'km' not in l.lower()), None)

        # 4) “modelo/etiqueta” (última línea)
        modelo_line = lines[-1] if lines else None

        detalle_auto.append(detalle_text)
        combustible_auto.append(combustible)
        transmision_auto.append(tipo_transmision)
        caja_auto.append(caja)
        kilometraje_auto.append(kilometraje_km)
        ubicacion_auto.append(ubicacion)
    
    ## URL DE AUTO
    
    url_auto = []
    enlace = driver.find_elements(By.CLASS_NAME, "c-results__link")   
    for link in enlace:
        url = link.get_attribute("href")    
        url_auto.append(url)
    
    ## ALMACENAMIENTO DE DATOS
    
    zip_list = zip(titulo_auto, marca_auto, modelo_auto, año_auto, precio_auto, tags, detalle_auto, combustible_auto, transmision_auto, caja_auto, kilometraje_auto, ubicacion_auto, url_auto)
    
    data_auto_page = []
    for titulo_auto_, marca_auto_, modelo_auto_, año_auto_, precio_auto_, tags_, detalle_auto_, combustible_auto_, transmision_auto_, caja_auto_, kilometraje_auto_, ubicacion_auto_, url_auto_ in zip_list:
        data_auto_page.append({
            "titulo": titulo_auto_,
            "tipo_vehiculo": tipo_vehiculo,
            "marca": marca_auto_,
            "modelo": modelo_auto_,
            "año": año_auto_,
            "precio": precio_auto_,
            "detalle": detalle_auto_,
            "combustible": combustible_auto_,
            "transmision": transmision_auto_,
            "caja": caja_auto_,
            "kilometraje": kilometraje_auto_,
            "ubicacion": ubicacion_auto_,
            "url_auto":url_auto_,
            "tags": tags_,
            "tipo": tipo,
            "pagina": link_pagina
        })
        
    neo_autos.extend(data_auto_page)
    

# driver.quit()
    
# print(f"Se extrajeorn informacion de {len(neo_autos)} unidades")
    

Páginas procesadas: 100%|██████████| 6/6 [00:19<00:00,  3.23s/vehiculo]


In [132]:
print(f"Se extrajeorn informacion de {len(neo_autos)} unidades")

Se extrajeorn informacion de 81 unidades


In [133]:
neo_autos

[{'titulo': 'Hyundai Grand I10 2024',
  'tipo_vehiculo': 'autos',
  'marca': 'Hyundai',
  'modelo': 'Grand I10',
  'año': '2024',
  'precio': 'US$ 13,490',
  'detalle': 'Gas GLP|Mecánica\nLima, Lima\nAUTO 0 KM A PRECIO DE SEMINUEVO VERSIÓN FULL GLP',
  'combustible': 'Gas Glp',
  'transmision': 'Mecánica',
  'caja': None,
  'kilometraje': 0,
  'ubicacion': 'Lima, Lima',
  'url_auto': 'https://neoauto.com/auto/nuevo/hyundai-grand-i10-2024-1846393',
  'tags': 'PREMIUM',
  'tipo': 'nuevos',
  'pagina': 'https://neoauto.com/venta-de-autos-nuevos?page=1'},
 {'titulo': 'Honda Pilot 2025',
  'tipo_vehiculo': 'autos',
  'marca': 'Honda',
  'modelo': 'Pilot',
  'año': '2025',
  'precio': 'US$ 57,990',
  'detalle': 'Gasolina|Automática\nLima, Lima\nVehículos 0KM En Honda Lima Norte',
  'combustible': 'Gasolina',
  'transmision': 'Automática',
  'caja': None,
  'kilometraje': 0,
  'ubicacion': 'Lima, Lima',
  'url_auto': 'https://neoauto.com/auto/nuevo/honda-pilot-2025-1805807',
  'tags': 'PREMIU

In [134]:
neo_autos[71]

{'titulo': 'Honda Type R 2024',
 'tipo_vehiculo': 'autos',
 'marca': 'Honda',
 'modelo': 'Type R',
 'año': '2024',
 'precio': 'US$ 56,990',
 'detalle': 'Gasolina|Mecánica\nLima, Lima\nCompra Tu Próximo 0KM En Maquinarias Honda',
 'combustible': 'Gasolina',
 'transmision': 'Mecánica',
 'caja': None,
 'kilometraje': 0,
 'ubicacion': 'Lima, Lima',
 'url_auto': 'https://neoauto.com/auto/nuevo/honda-type-r-2024-1827045',
 'tags': 'PREMIUM',
 'tipo': 'nuevos',
 'pagina': 'https://neoauto.com/venta-de-autos-nuevos?page=6'}

In [135]:
neo_autos_df = pd.DataFrame(neo_autos)
neo_autos_df

,titulo,tipo_vehiculo,marca,modelo,año,precio,detalle,combustible,transmision,caja,kilometraje,ubicacion,url_auto,tags,tipo,pagina
0,Hyundai Grand I10 2024,autos,Hyundai,Grand I10,2024,"US$ 13,490","Gas GLP|Mecánica\nLima, Lima\nAUTO 0 KM A PREC...",Gas Glp,Mecánica,None,0.0,"Lima, Lima",https://neoauto.com/auto/nuevo/hyundai-grand-i...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1
1,Honda Pilot 2025,autos,Honda,Pilot,2025,"US$ 57,990","Gasolina|Automática\nLima, Lima\nVehículos 0KM...",Gasolina,Automática,None,0.0,"Lima, Lima",https://neoauto.com/auto/nuevo/honda-pilot-202...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1
2,Hyundai Creta Grand 2025,autos,Hyundai,Creta Grand,2025,"US$ 25,990","Gas GLP|Automática\nLima, Lima\nGARANTÍA VIGEN...",Gas Glp,Automática,None,NaN,"Lima, Lima",https://neoauto.com/auto/nuevo/hyundai-creta-g...,MOTOR POTENTE,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=2
3,Ford F-150 2024,autos,Ford,F-150,2024,"US$ 50,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,0.0,"Lima, Lima",https://neoauto.com/auto/nuevo/ford-f-150-2024...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=2
4,Honda Accord 2025,autos,Honda,Accord,2025,"US$ 43,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,0.0,"Lima, Lima",https://neoauto.com/auto/nuevo/honda-accord-20...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Dfsk Glory 330s 2025,autos,DFSK,Glory 330s,2025,"US$ 13,790","Gasolina|Mecánica\nLima, Lima\nCompra Tu Próxi...",Gasolina,Mecánica,None,0.0,"Lima, Lima",https://neoauto.com/auto/nuevo/dfsk-glory-330s...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6
77,Honda Zr-V 2025,autos,Honda,Zr-V,2025,"US$ 33,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,0.0,"Lima, Lima",https://neoauto.com/auto/nuevo/honda-zr-v-2025...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6
78,Mazda Mx 5 2025,autos,Mazda,Mx 5,2025,"US$ 43,490","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,0.0,"Lima, Lima",https://neoauto.com/auto/nuevo/mazda-mx-5-2025...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6
79,Renault Master Furgon 2026,autos,Renault,Master Furgon,2026,"US$ 40,490","Diesel|Mecánica\nLima, Lima\nCompra Tu Próximo...",Diesel,Mecánica,None,0.0,"Lima, Lima",https://neoauto.com/auto/nuevo/renault-master-...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6


In [136]:
neo_autos_df["pagina"].value_counts()

pagina
https://neoauto.com/venta-de-autos-nuevos?page=4    18
https://neoauto.com/venta-de-autos-nuevos?page=6    17
https://neoauto.com/venta-de-autos-nuevos?page=3    16
https://neoauto.com/venta-de-autos-nuevos?page=2    16
https://neoauto.com/venta-de-autos-nuevos?page=5    12
https://neoauto.com/venta-de-autos-nuevos?page=1     2
Name: count, dtype: int64

In [119]:
neo_autos_df[["titulo", "marca", "modelo", "año", "precio", "url_auto"]]

,titulo,marca,modelo,año,precio,url_auto
0,Hyundai Grand I10 2024,Hyundai,Grand I10,2024,"US$ 13,490",https://neoauto.com/auto/nuevo/hyundai-grand-i...
1,Honda Pilot 2025,Honda,Pilot,2025,"US$ 57,990",https://neoauto.com/auto/nuevo/honda-pilot-202...
2,Hyundai Creta Grand 2025,Hyundai,Creta Grand,2025,"US$ 25,990",https://neoauto.com/auto/nuevo/hyundai-creta-g...
3,Ford F-150 2024,Ford,F-150,2024,"US$ 50,990",https://neoauto.com/auto/nuevo/ford-f-150-2024...
4,Honda Accord 2025,Honda,Accord,2025,"US$ 43,990",https://neoauto.com/auto/nuevo/honda-accord-20...
...,...,...,...,...,...,...
76,Dfsk Glory 330s 2025,DFSK,Glory 330s,2025,"US$ 13,790",https://neoauto.com/auto/nuevo/dfsk-glory-330s...
77,Honda Zr-V 2025,Honda,Zr-V,2025,"US$ 33,990",https://neoauto.com/auto/nuevo/honda-zr-v-2025...
78,Mazda Mx 5 2025,Mazda,Mx 5,2025,"US$ 43,490",https://neoauto.com/auto/nuevo/mazda-mx-5-2025...
79,Renault Master Furgon 2026,Renault,Master Furgon,2026,"US$ 40,490",https://neoauto.com/auto/nuevo/renault-master-...


In [120]:
# normalizaciones básicas
neo_autos_df["url_low"]     = neo_autos_df["url_auto"].str.lower()
neo_autos_df["marca_slug"]  = neo_autos_df["marca"].str.lower().str.replace(r"[^a-z0-9]+","-", regex=True)
neo_autos_df["modelo_slug"] = neo_autos_df["modelo"].str.lower().str.replace(r"[^a-z0-9]+","-", regex=True)

# checks fila a fila (sin regex fancy)
neo_autos_df["brand_in_url"] = neo_autos_df.apply(lambda r: r["marca_slug"]  in r["url_low"], axis=1)
neo_autos_df["model_in_url"] = neo_autos_df.apply(lambda r: r["modelo_slug"] in r["url_low"], axis=1)
neo_autos_df["year_in_url"]  = neo_autos_df.apply(lambda r: str(r["año"]) in r["url_low"], axis=1)  # usa "anio" si tu col se llama así

neo_autos_df["url_ok"] = neo_autos_df[["brand_in_url","model_in_url","year_in_url"]].all(axis=1)

In [121]:
neo_autos_df

,titulo,tipo_vehiculo,marca,modelo,año,precio,detalle,combustible,transmision,caja,...,tags,tipo,pagina,url_low,marca_slug,modelo_slug,brand_in_url,model_in_url,year_in_url,url_ok
0,Hyundai Grand I10 2024,autos,Hyundai,Grand I10,2024,"US$ 13,490","Gas GLP|Mecánica\nLima, Lima\nAUTO 0 KM A PREC...",Gas Glp,Mecánica,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1,https://neoauto.com/auto/nuevo/hyundai-grand-i...,hyundai,grand-i10,True,True,True,True
1,Honda Pilot 2025,autos,Honda,Pilot,2025,"US$ 57,990","Gasolina|Automática\nLima, Lima\nVehículos 0KM...",Gasolina,Automática,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1,https://neoauto.com/auto/nuevo/honda-pilot-202...,honda,pilot,True,True,True,True
2,Hyundai Creta Grand 2025,autos,Hyundai,Creta Grand,2025,"US$ 25,990","Gas GLP|Automática\nLima, Lima\nGARANTÍA VIGEN...",Gas Glp,Automática,None,...,MOTOR POTENTE,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=2,https://neoauto.com/auto/nuevo/hyundai-creta-g...,hyundai,creta-grand,True,True,True,True
3,Ford F-150 2024,autos,Ford,F-150,2024,"US$ 50,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=2,https://neoauto.com/auto/nuevo/ford-f-150-2024...,ford,f-150,True,True,True,True
4,Honda Accord 2025,autos,Honda,Accord,2025,"US$ 43,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=2,https://neoauto.com/auto/nuevo/honda-accord-20...,honda,accord,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Dfsk Glory 330s 2025,autos,DFSK,Glory 330s,2025,"US$ 13,790","Gasolina|Mecánica\nLima, Lima\nCompra Tu Próxi...",Gasolina,Mecánica,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6,https://neoauto.com/auto/nuevo/dfsk-glory-330s...,dfsk,glory-330s,True,True,True,True
77,Honda Zr-V 2025,autos,Honda,Zr-V,2025,"US$ 33,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6,https://neoauto.com/auto/nuevo/honda-zr-v-2025...,honda,zr-v,True,True,True,True
78,Mazda Mx 5 2025,autos,Mazda,Mx 5,2025,"US$ 43,490","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",Gasolina,Automática,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6,https://neoauto.com/auto/nuevo/mazda-mx-5-2025...,mazda,mx-5,True,True,True,True
79,Renault Master Furgon 2026,autos,Renault,Master Furgon,2026,"US$ 40,490","Diesel|Mecánica\nLima, Lima\nCompra Tu Próximo...",Diesel,Mecánica,None,...,PREMIUM,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6,https://neoauto.com/auto/nuevo/renault-master-...,renault,master-furgon,True,True,True,True


In [123]:
neo_autos_df[neo_autos_df["titulo"]=="Hyundai Creta 2026"][["titulo", "precio", "url_auto", "pagina"]]

,titulo,precio,url_auto,pagina
53,Hyundai Creta 2026,"US$ 24,290",https://neoauto.com/auto/nuevo/hyundai-creta-2...,https://neoauto.com/venta-de-autos-nuevos?page=5


In [124]:
# ver sospechosos
sospechosos = neo_autos_df.loc[~neo_autos_df["url_ok"], ["titulo","marca","modelo","año","url_auto","brand_in_url","model_in_url","year_in_url"]]
sospechosos

,titulo,marca,modelo,año,url_auto,brand_in_url,model_in_url,year_in_url


In [145]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")   # modo oculto
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
# options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options = options)
# driver.minimize_window()

# --- Config mínimo ---
YEAR_RX  = re.compile(r'\b(19[8-9]\d|20[0-3]\d)\b')
NOISE_RX = re.compile(r'\bauto\s+nuev[oa]s?\b', re.I)
BRANDS = sorted([
    "Alfa Romeo","Aston Martin","Audi","Bentley","BMW","Citroen","Cupra","Dacia","Ferrari","Fiat",
    "Jaguar","Lamborghini","Land Rover","Maserati","Maybach","McLaren","Mercedes Benz","Mini","Opel",
    "Peugeot","Porsche","Renault","Rolls Royce","Seat","Skoda","Smart","Volkswagen","Volvo",
    "Acura","Daihatsu","Honda","Infiniti","Isuzu","Lexus","Mazda","Mitsubishi","Nissan","Subaru",
    "Suzuki","Toyota","Hyundai","Genesis","Kia","SsangYong","Buick","Cadillac","Chevrolet","Chrysler",
    "Dodge","Ford","GMC","Hummer","Jeep","Lincoln","Ram","Tesla","BAIC","BYD","Changan","Chery","DFSK",
    "Dongfeng","FAW","Foton","Geely","Great Wall","Haval","JAC","Jetour","Maxus","MG","Omoda","Zotye","JMC",
], key=len, reverse=True)

def clean_title(s):  # quita "AUTO NUEVO", espacios dobles
    return re.sub(r'\s+', ' ', NOISE_RX.sub('', s)).strip()

def parse_brand_model_year(t):
    t = clean_title(t)
    # año: último match si hay varios
    ys = YEAR_RX.findall(t); anio = ys[-1] if ys else None
    # marca: primera que aparezca (BRANDS ya está ordenado por longitud desc)
    tl = t.lower()
    marca = next((b for b in BRANDS if b.lower() in tl), "Otra Marca")
    # modelo: t sin marca ni año
    modelo = re.sub(re.escape(marca), '', t, flags=re.I).strip() if marca != "Otra Marca" else t
    if anio: modelo = re.sub(rf'\b{anio}\b', '', modelo).strip()
    return t, marca, modelo, anio

neo_autos = []

for pagina in paginas:  # pagina = {"url": ..., "tipo": ..., "tipo_vehiculo": ...}
    driver.get(pagina["url"])

    # 1) espera a que aparezcan resultados
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.c-results__link")))

    # 2) scroll simple (3–4 bajadas) para forzar lazy-load
    prev = 0
    for _ in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.2)
        n = len(driver.find_elements(By.CSS_SELECTOR, "a.c-results__link"))
        if n == prev: break
        prev = n

    # 3) recorre por tarjeta (evita desalineaciones)
    cards = driver.find_elements(By.CSS_SELECTOR, "a.c-results__link")
    vistos = set()

    for a in cards:
        url = a.get_attribute("href")
        if not url or url in vistos: 
            continue
        vistos.add(url)

        # sube al contenedor de la card (si falla, usamos el link como fallback)
        try:
            card = a.find_element(By.XPATH, "./ancestor::*[self::li or self::article][1]")
        except:
            card = a

        # título
        try:
            titulo_raw = card.find_element(By.CSS_SELECTOR, ".c-results__header").text
        except:
            titulo_raw = ""

        titulo, marca, modelo, anio = parse_brand_model_year(titulo_raw)

        # precio (si no hay, None)
        try:
            precio = card.find_element(By.CSS_SELECTOR, ".c-results-mount__price").text
        except:
            precio = None

        # detalle (opcional)
        try:
            detalle = card.find_element(By.CSS_SELECTOR, ".c-results-details__description").text
        except:
            detalle = ""

        # tags (opcionales)
        tags = [e.text for e in card.find_elements(By.CSS_SELECTOR, ".c-results-tag__stick") if e.text.strip()]

        neo_autos.append({
            "titulo": titulo,
            "tipo_vehiculo": pagina["tipo_vehiculo"],
            "marca": marca,
            "modelo": modelo,
            "año": anio,
            "precio": precio,
            "detalle": detalle,
            "tags": tags,
            "url_auto": url,
            "tipo": pagina["tipo"],
            "pagina": pagina["url"]
        })


In [146]:
len(neo_autos)

118

In [147]:
neo_autos_df = pd.DataFrame(neo_autos)
neo_autos_df

,titulo,tipo_vehiculo,marca,modelo,año,precio,detalle,tags,url_auto,tipo,pagina
0,Hyundai Grand I10 2024,autos,Hyundai,Grand I10,2024,"US$ 13,490","Gas GLP|Mecánica\nLima, Lima\nAUTO 0 KM A PREC...",[],https://neoauto.com/auto/nuevo/hyundai-grand-i...,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1
1,Honda Pilot 2025,autos,Honda,Pilot,2025,"US$ 57,990","Gasolina|Automática\nLima, Lima\nVehículos 0KM...",[PREMIUM],https://neoauto.com/auto/nuevo/honda-pilot-202...,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1
2,Hyundai Tucson 2024,autos,Hyundai,Tucson,2024,"US$ 29,990","Gasolina|Automática\nLima, Lima\nGARANTÍA VIGE...",[],https://neoauto.com/auto/nuevo/hyundai-tucson-...,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1
3,Hyundai Tucson 2024,autos,Hyundai,Tucson,2024,"US$ 29,990","Gasolina|Automática - Secuencial\nLima, Lima\n...",[],https://neoauto.com/auto/nuevo/hyundai-tucson-...,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1
4,Hyundai Grand I10 2024,autos,Hyundai,Grand I10,2024,"US$ 13,490","Gas GLP|Mecánica\nLima, Lima\nAUTO 0 KM A PREC...",[],https://neoauto.com/auto/nuevo/hyundai-grand-i...,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=1
...,...,...,...,...,...,...,...,...,...,...,...
113,Honda Zr-V 2025,autos,Honda,Zr-V,2025,"US$ 33,990","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",[PREMIUM],https://neoauto.com/auto/nuevo/honda-zr-v-2025...,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6
114,Mazda Mx 5 2025,autos,Mazda,Mx 5,2025,"US$ 43,490","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",[PREMIUM],https://neoauto.com/auto/nuevo/mazda-mx-5-2025...,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6
115,Renault Master Furgon 2026,autos,Renault,Master Furgon,2026,"US$ 40,490","Diesel|Mecánica\nLima, Lima\nCompra Tu Próximo...",[PREMIUM],https://neoauto.com/auto/nuevo/renault-master-...,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6
116,Nissan Patrol 2024,autos,Nissan,Patrol,2024,"US$ 84,800","Gasolina|Automática\nLima, Lima\nCompra Tu Pró...",[PREMIUM],https://neoauto.com/auto/nuevo/nissan-patrol-2...,nuevos,https://neoauto.com/venta-de-autos-nuevos?page=6


In [148]:
neo_autos_df["pagina"].value_counts()

pagina
https://neoauto.com/venta-de-autos-nuevos?page=1    20
https://neoauto.com/venta-de-autos-nuevos?page=2    20
https://neoauto.com/venta-de-autos-nuevos?page=3    20
https://neoauto.com/venta-de-autos-nuevos?page=4    20
https://neoauto.com/venta-de-autos-nuevos?page=5    20
https://neoauto.com/venta-de-autos-nuevos?page=6    18
Name: count, dtype: int64

In [149]:
neo_autos_df["marca"].value_counts()

marca
Hyundai    31
Kia        17
Honda      13
Mazda      13
Ford       12
Nissan      9
DFSK        8
JMC         7
Renault     7
Mini        1
Name: count, dtype: int64